In [1]:
import pandas as pd
import sqlite3
import time

def analysiere_testdatei_datenbank(test_datei, verbindung):
    """
    Analysiert eine einzelne Testdatei mit optimierten SQL-Abfragen.
    """
    start_zeit = time.time()

    # Testdaten laden und in SQLite speichern
    test_daten = pd.read_csv(test_datei, sep=";", decimal=",")
    test_daten.to_sql("PruefungsDaten", verbindung, if_exists="replace", index=False)

    # SQL-Abfrage zur Verarbeitung der Referenzdaten
    sql_verarbeitung = """
        WITH ReferenzGruppiert AS (
            SELECT 
                Patient, Position, Wave, Befall, 
                AVG(Value) AS Mittelwert, COUNT(Value) AS Anzahl, 
                SUM(Value * Value) AS SummeQuadrate
            FROM ReferenzDaten
            WHERE Art = 'Gewebe' AND Value >= 0
            GROUP BY Patient, Position, Wave, Befall
        ),
        BerechneteVarianz AS (
            SELECT 
                Patient, Position, Wave, Befall, Mittelwert, 
                (SummeQuadrate - Anzahl * Mittelwert * Mittelwert) / NULLIF(Anzahl, 0) AS Varianz
            FROM ReferenzGruppiert
        ),
        GefilterteReferenzen AS (
            SELECT 
                Patient, Position, Wave, Befall, Mittelwert, 
                SQRT(Varianz) AS Standardabweichung
            FROM BerechneteVarianz
            WHERE Varianz IS NOT NULL
        ),
        BereinigteReferenzen AS (
            SELECT 
                r.Patient, r.Position, r.Wave, r.Befall, 
                AVG(r.Value) AS Durchschnittswert
            FROM ReferenzDaten r
            INNER JOIN GefilterteReferenzen f
            ON r.Patient = f.Patient 
               AND r.Position = f.Position 
               AND r.Wave = f.Wave 
               AND r.Befall = f.Befall
            WHERE ABS(r.Value - f.Mittelwert) <= 2 * f.Standardabweichung
            GROUP BY r.Patient, r.Position, r.Wave, r.Befall
        )
        SELECT 
            t.wave AS TestWave, 
            r.Wave AS RefWave, 
            r.Durchschnittswert, 
            t.value,
            CASE
                WHEN t.value BETWEEN r.Durchschnittswert * 0.5 AND r.Durchschnittswert * 1.5 THEN 'N'
                ELSE 'T'
            END AS BefallErgebnis
        FROM PruefungsDaten t
        LEFT JOIN BereinigteReferenzen r
        ON ABS(t.wave - r.Wave) <= 0.5;
    """
    ergebnisse = pd.read_sql(sql_verarbeitung, verbindung)

    # Ergebnisse speichern
    ergebnisse.to_csv(f"{test_datei}_analyse.csv", sep=";", decimal=",", index=False)

    # SQL-Abfrage für aggregierte Ergebnisse
    sql_aggregierte_ergebnisse = """
        WITH AggregierteErgebnisse AS (
            SELECT 
                TestWave, 
                COUNT(CASE WHEN BefallErgebnis = 'N' THEN 1 ELSE NULL END) AS N_Anzahl,
                COUNT(*) AS GesamtAnzahl
            FROM (
                SELECT 
                    t.wave AS TestWave, 
                    r.Wave AS RefWave, 
                    r.Durchschnittswert, 
                    t.value,
                    CASE
                        WHEN t.value BETWEEN r.Durchschnittswert * 0.5 AND r.Durchschnittswert * 1.5 THEN 'N'
                        ELSE 'T'
                    END AS BefallErgebnis
                FROM PruefungsDaten t
                LEFT JOIN BereinigteReferenzen r
                ON ABS(t.wave - r.Wave) <= 0.5
            )
            GROUP BY TestWave
        )
        SELECT 
            TestWave, 
            N_Anzahl * 1.0 / GesamtAnzahl AS N_Prozent,
            CASE
                WHEN N_Anzahl * 1.0 / GesamtAnzahl >= 0.7 THEN 'N'
                ELSE 'T'
            END AS Endergebnis
        FROM AggregierteErgebnisse;
    """
    final_ergebnisse = pd.read_sql(sql_aggregierte_ergebnisse, verbindung)

    # Aggregierte Ergebnisse speichern
    final_ergebnisse.to_csv(f"{test_datei}_finale_ergebnisse.csv", sep=";", decimal=",", index=False)

    dauer = time.time() - start_zeit
    print(f"Analyse von {test_datei} abgeschlossen in {dauer:.2f} Sekunden.")

def hauptanalyse():
    """
    Hauptfunktion zur Verarbeitung der Referenz- und Testdaten.
    """
    # Referenzdaten laden und in SQLite speichern
    referenz_daten = pd.read_csv("LaserBase.csv", sep=";", decimal=",")
    verbindung = sqlite3.connect(":memory:")
    referenz_daten.to_sql("ReferenzDaten", verbindung, if_exists="replace", index=False)

    # Indizes erstellen, um die SQL-Abfragen zu optimieren
    verbindung.execute("CREATE INDEX idx_wave ON ReferenzDaten(Wave);")
    verbindung.execute("CREATE INDEX idx_patient_position ON ReferenzDaten(Patient, Position);")

    # Testdateien analysieren
    test_dateien = ["LaserBaseTest1.csv", "LaserBaseTest2.csv", "LaserBaseTest3.csv"]
    for test_datei in test_dateien:
        analysiere_testdatei_datenbank(test_datei, verbindung)

    # Verbindung schließen
    verbindung.close()

if __name__ == "__main__":
    hauptanalyse()


DatabaseError: Execution failed on sql '
        WITH ReferenzGruppiert AS (
            SELECT 
                Patient, Position, Wave, Befall, 
                AVG(Value) AS Mittelwert, COUNT(Value) AS Anzahl, 
                SUM(Value * Value) AS SummeQuadrate
            FROM ReferenzDaten
            WHERE Art = 'Gewebe' AND Value >= 0
            GROUP BY Patient, Position, Wave, Befall
        ),
        BerechneteVarianz AS (
            SELECT 
                Patient, Position, Wave, Befall, Mittelwert, 
                (SummeQuadrate - Anzahl * Mittelwert * Mittelwert) / NULLIF(Anzahl, 0) AS Varianz
            FROM ReferenzGruppiert
        ),
        GefilterteReferenzen AS (
            SELECT 
                Patient, Position, Wave, Befall, Mittelwert, 
                SQRT(Varianz) AS Standardabweichung
            FROM BerechneteVarianz
            WHERE Varianz IS NOT NULL
        ),
        BereinigteReferenzen AS (
            SELECT 
                r.Patient, r